In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd

Update the equity index values from 2024 to 2025

In [3]:
df_equity_2025 = pd.read_csv('../ct-data/equity-2025.csv', encoding='latin')
gdf_equity_map_2024 = gpd.read_file('../ct-data/equity-2024-layer.geo.json')

In [4]:
# Force both keys to the same string format with 2 decimal places
df_equity_2025["Census tract"] = df_equity_2025["Census tract"].astype(float).map("{:.2f}".format)
gdf_equity_map_2024["ctuid"] = gdf_equity_map_2024["ctuid"].astype(float).map("{:.2f}".format)

# Create mapping
score_map = df_equity_2025.set_index("Census tract")["Updated Equity Score"]

# Replace values
gdf_equity_map_2024["Equity Index"] = gdf_equity_map_2024["ctuid"].map(score_map).fillna(
    gdf_equity_map_2024["Equity Index"]
)

gdf_equity_map_2024.to_file("../ct-data/ct-data-all.geo.json", driver="GeoJSON")

Identify the breakpoints for quintiles

In [5]:
gdf_ct = gpd.read_file('../ct-data/ct-data-all.geo.json')

# Calculate quintile breakpoints
quintiles = np.quantile(gdf_ct['Equity Index'].dropna(), [0, 0.2, 0.4, 0.6, 0.8, 1])
print("Quintile breakpoints:", quintiles)

Quintile breakpoints: [0.   0.3  0.37 0.44 0.53 1.  ]
